# 20240620
---

## 최빈수 구하기

In [1]:
import numpy as np

x = np.array([1, 2, 1, 3, 1])
mode = np.bincount(x).argmax()

In [5]:
from scipy.stats import mode

x = np.array([1, 2, 1, 1, 3, 1, 4, 4, 5, 6, 6, 6, 7, 7, 6, 6])
result = mode(x)
print(result.mode)

6


## 빈도수 구하기

In [7]:
import pandas as pd

df = pd.read_csv('./datasets/mtcars.csv')

result = pd.Series(df['cyl']).value_counts()
print(result)

cyl
8    14
4    11
6     7
Name: count, dtype: int64


## 제1유형
---

In [45]:
import numpy as np
import pandas as pd

df = pd.read_csv('./datasets/P210302.csv')

total_count = len(df)
missing_value_count = df.isna().sum()

ratio = missing_value_count / total_count

ratio = pd.DataFrame(ratio)
ratio = ratio.sort_values(by=0, ascending=False)

print(ratio.index[0])

Age


## 제2유형
---

## 제3유형
---

### Z-검정

- 모평균, 모표준편차 확인 가능할 경우

In [11]:
import numpy as np
from scipy.stats import norm

x = np.array([25, 27, 31, 23, 24, 30, 26])

mean_x = np.mean(x)   # 표본평균
mean_mu = 26
sigma = 5    # 모표준편차
n = len(x)   # 표본 크기

# z = (표본평균 - 모평균) / (모표준편차 / root(표본크기))
z = (mean_x - mean_mu) / (sigma / np.sqrt(n))
print(f"z-값: {z}")

p = (1 - norm.cdf(z)) * 2
print(f"p-값: {p}")

z-값: 0.3023715784073826
p-값: 0.7623688184698392


### T-검정

- 독립변수 : 범주형, 종속변수 : 수치형
- 두 집단의 평균 비교
- 표본이 정규성, 등분산성, 독립성을 만족할 경우 적용

#### (1) 단일 표본 T-검정

- 한 집단의 평균(표본평균)이 모집단의 평균(모평균)과 같은지 검정하는 방법
- 검정을 수행하기 전에 샤피로 윌크 검정을 이용하여 **정규성 검정** 을 진행한다.
    - 정규성을 만족할 경우 `ttest_1samp()` 함수를 사용하여 검정을 수행한다.
    - 정규성을 만족하지 않을 경우 `wilcoxon()` 함수를 사용하여 검정을 수행한다.

In [12]:
import numpy as np
from scipy.stats import shapiro

np.random.seed(123)
x = np.random.normal(loc=0, scale=1, size=50)

# 정규성 검정 수행
result = shapiro(x)
print(result)   # p-값이 0.05 보다 크므로 귀무가설 채택 (정규성 가정 만족)

ShapiroResult(statistic=0.98505924025743, pvalue=0.774149900641557)


> 정규성 가정을 만족할 경우

In [14]:
import pandas as pd
from scipy.stats import shapiro, ttest_1samp

df = pd.DataFrame({
    'height': [12, 14, 16, 11, 17, 13]
})

# 정규성 검정
result = shapiro(df['height'])
print(result)   # p-값이 0.05보다 크므로 정규성 가정 만족

result = ttest_1samp(df['height'], popmean=11)
print(result)   # p-값이 0.05보다 작으므로 대립가설 선택

ShapiroResult(statistic=0.9575422062677347, pvalue=0.8006123833637463)
TtestResult(statistic=2.9958563516135266, pvalue=0.03024293910122357, df=5)


> 정규성 가정을 만족하지 않을 경우

In [16]:
import pandas as pd
from scipy.stats import shapiro, wilcoxon

df = pd.read_csv('./datasets/cats.csv')

# 정규성 검정
target = df['Bwt']
result = shapiro(target)
print(result)    # p-value가 0.05 보다 작으므로 정규성 가정 불만족

# 윌콕슨 검정
result = wilcoxon(target, alternative='two-sided')
print(result)   # p-값이 0.05 보다 작으므로 대립가설 채택

ShapiroResult(statistic=0.9518791269479144, pvalue=6.730857622701013e-05)
WilcoxonResult(statistic=0.0, pvalue=2.096734736413612e-25)


#### (2) 쌍체 표본 T-검정

- 한 집단에서 처치를 받기 전과 후의 차이를 알아보기 위해 사용
- 표본이 하나, 독립 변수가 1개일 때 사용
- 종속형 변수는 연속형 변수이어야 한다.
- 모집단의 관측값이 정규성 가정을 만족해야 한다.

In [20]:
import pandas as pd
from scipy.stats import ttest_rel, shapiro

data = pd.DataFrame({
    'before': [5, 3, 8, 4, 3, 2, 1],
    'after': [8, 6, 6, 5, 8, 7, 3]
})

# 쌍체 표본 검정 (정규성 가정 만족한다는 가정 하에)
result = ttest_rel(data['before'], data['after'], alternative='less')
print(result)

TtestResult(statistic=-2.633628675421043, pvalue=0.019435182851729293, df=6)


#### (3) 독립 표본 T-검정

- 데이터가 서로 다른 모집단에서 추출된 경우 사용
- 독립된 두 집단의 평균 차이를 검정하는 방법
- 검정 전 반드시 정규성, 등분산성 가정을 만족하는지 확인
- 독립 변수는 범주형, 종속 변수는 연속형
- 검정 전 **등분산성 검정** 을 수행한다.

In [23]:
import pandas as pd
from scipy.stats import levene, ttest_ind

df = pd.read_csv('./datasets/cats.csv')

# 그룹 지정
group1 = df[df['Sex'] == 'F']['Bwt']
group2 = df[df['Sex'] == 'M']['Bwt']

# 등분산성 검정
result = levene(group1, group2)
print(result)   # p-값이 0.05보다 작으므로 대립가설 채택 (등분산성 불만족)

# 독립 표본 T-검정
result = ttest_ind(group1, group2, equal_var=False)
print(result)   # p-값이 0.05 보다 작으므로 대립가설 채택

LeveneResult(statistic=19.43101190877999, pvalue=2.0435285255189404e-05)
TtestResult(statistic=-8.70948849909559, pvalue=8.831034455859356e-15, df=136.83788299625363)


### F-검정

- 두 표본의 분산에 대한 차이가 통계적으로 유의미한지 검정
- 두 모집단 간의 비율에 대한 검정

In [26]:
import numpy as np
from scipy.stats import f

df1 = np.array([1, 2, 3, 4, 6])
df2 = np.array([4, 5, 6, 7, 8])

var1 = np.var(df1)
var2 = np.var(df2)


# F = x_var / y_var (x_var > y_var)
def f_test(x, y):

    x_var = np.var(x)
    y_var = np.var(y)

    if x_var < y_var:
        x, y = y, x
    
    f_value = x_var / y_var
    x_dof = x.size - 1
    y_dof = y.size - 1

    # F-검정
    p_value = (1 - f.cdf(f_value, x_dof, y_dof)) * 2   # 양측 검정

    return f_value, p_value


result = f_test(df1, df2)
print(result)


(1.48, 0.7133026753046221)


### 카이제곱 검정

- 범주형 자료 간의 차이를 보여주는 기법
- 관측 빈도가 기대 빈도와 유의하게 다른지 검정하는 방법
- 적합도 검정, 독립성 검정, 동질성 검정 3가지 방법 존재

#### (1) 적합도 검정

- 표본 집단이 주어진 분포를 따르고 있는지 검정하는 방법
- 구분 범주가 상호 베타적이어야 함. (예: 성별(남자, 여자), 등수(1등, 2등, 3등))
- 자유도 : (범주의 수) - 1

In [27]:
import numpy as np
from scipy.stats import chisquare

observe = np.array([90, 160])
expect = np.array([0.45, 0.55]) * np.sum(observe)

# 적합도 검정
result = chisquare(f_obs=observe, f_exp=expect)
print(result)   # p-값이 0.05 보다 작으므로 대립가설 채택

Power_divergenceResult(statistic=8.181818181818182, pvalue=0.004231232899758152)


#### (2) 독립성 검정

- 범주가 2개 이상의 범주로 분할되어 있을 때 사용
- 각 범주가 서로 독립적인지, 서로 연관성이 있는지 검정하는 방법
    - 예) 학년(1학년, 2학년, 3학년)이라는 범주형 데이터와 선호 과목(국, 영, 수)이라는 범주형 데이터 간에 서로 연관성이 있는지 독립적인지를 판단하는 문제에서 사용
- 귀무 가설 : 요인1과 요인2는 독립적이다.
- 자유도 : {(범주1의 수) - 1} * {(범주2의 수) - 1}

In [30]:
import pandas as pd
from scipy.stats import chi2_contingency

df = pd.read_csv('./datasets/survey.csv')

# 교차표 형태로 만들기
table = pd.crosstab(df['Sex'], df['Exer'])

# 독립성 검정
result = chi2_contingency(table)
print(result)    # p-값이 0.05보다 작으므로 대립가설 채택 (서로 독립 X)


Chi2ContingencyResult(statistic=4.904232352768243, pvalue=0.0267909570897706, dof=1, expected_freq=array([[57.53773585, 49.46226415],
       [56.46226415, 48.53773585]]))


#### (3) 동질성 검정

- 각가의 독립적인 부모 집단으로부터 정해진 표본의 크기 만큼 자료를 추출하는 경우 관측값들이 정해진 범주 내에서 서로 동질한지 여부를 검정하는 기법
    - 예) 남학생과 여학생 그룹에 대하여 각 그룹이 선호하는 과목이 같은지 여부를 판단하는 문제
- 귀무 가설 : 모집단은 동질하다.
- 동질성 검정과 독립성 검정은 계산 방식은 동일하다.

In [32]:
import numpy as np
from scipy.stats import chi2_contingency

data = np.array([[10, 20, 30], [20, 20, 20], [30, 10, 10]])

result = chi2_contingency(data)

print(result)

Chi2ContingencyResult(statistic=23.799999999999997, pvalue=8.759622201519535e-05, dof=4, expected_freq=array([[21.17647059, 17.64705882, 21.17647059],
       [21.17647059, 17.64705882, 21.17647059],
       [17.64705882, 14.70588235, 17.64705882]]))
